# BGG Extractor Library Demo (Jupyter Notebook)

This notebook demonstrates the BGG Extractor library using async/await syntax.

## Prerequisites
1. Set `BGG_API_TOKEN` in your environment or `.env` file
2. Installed the package: `pip install -e .`

## 1. Import and Setup

In [1]:
import os

from dotenv import load_dotenv

from bgg_extractor import BGGClient, save_json
from bgg_extractor.transform import models_to_list

load_dotenv()
token = os.getenv("BGG_API_TOKEN")

print("✅ Setup complete!")

✅ Setup complete!


## 2. Search for Games

In [2]:
# Create client and search
async with BGGClient(token=token) as client:
    results = await client.search("Catan")

print(f"Found {len(results.items)} results")
print("\nFirst 5 results:")
for item in results.items[:5]:
    print(f"  - {item.name} (ID: {item.id})")

Found 312 results

First 5 results:
  - The 7 Wonders of Catan (fan expansion for Catan) (ID: 134277)
  - 7 Wonders: Catan (ID: 110308)
  - Baden-Württemberg Catan (ID: 123386)
  - Barna fra Catan (ID: 5824)
  - CATAN (ID: 13)


## 3. Get Game Details

In [3]:
# Get details for specific games (13 = Catan, 174430 = Gloomhaven)
# Fetch 100 games in batches of 20

game_ids = list(range(100))
batch_size = 20
all_games = []


async with BGGClient(token=token) as client:
    for i in range(0, len(game_ids), batch_size):
        batch = game_ids[i : i + batch_size]
        print(f"Fetching batch {i // batch_size + 1}...")
        result = await client.get_thing(batch, stats=True)
        all_games.extend(result.items)

print(f"Total games fetched: {len(all_games)}")

Fetching batch 1...
Fetching batch 2...
Fetching batch 3...
Fetching batch 4...
Fetching batch 5...
Total games fetched: 95


#### Run: for a list of all games
`all_games = bgg_client.get_all_games()`

In [ ]:
all_games

[ThingItem(id=None, type=None, name=None, description=None, yearpublished=None, minplayers=None, maxplayers=None, playingtime=None, minage=None, usersrated=None, average=None, rank=None, categories=[], mechanics=[], designers=[], artists=[], publishers=[]),
 ThingItem(id=1, type='boardgame', name='Die Macher', description='Die Macher is a game about seven sequential political races in different regions of Germany. Players are in charge of national political parties, and must manage limited resources to help their party to victory. The winning party will have the most victory points after all the regional elections. There are four different ways of scoring victory points. First, each regional election can supply one to eighty victory points, depending on the size of the region and how well your party does in it. Second, if a party wins a regional election and has some media influence in the region, then the party will receive some media-control victory points. Third, each party has a na

## Storing Pydantic objects can be memory-intensive. Here are better approaches:

### Option 1: Convert to Dicts Immediately (Lighter)

In [5]:
# Convert to dicts as you fetch - much lighter than Pydantic models
game_ids = list(range(1, 101))
batch_size = 20
all_games = []  # Will store dicts, not Pydantic models

async with BGGClient(token=token) as client:
    for i in range(0, len(game_ids), batch_size):
        batch = game_ids[i : i + batch_size]
        print(f"Fetching batch {i // batch_size + 1}...")
        result = await client.get_thing(batch, stats=True)
        # Convert to dicts immediately
        all_games.extend([game.model_dump() for game in result.items])

print(f"Total games fetched: {len(all_games)}")

Fetching batch 1...
Fetching batch 2...
Fetching batch 3...
Fetching batch 4...
Fetching batch 5...
Total games fetched: 94


### Option 2: Stream to Disk (Best for large datasets)

In [ ]:
# Save batches to disk, don't hold in memory
import json

game_ids = list(range(1, 1001))
batch_size = 20
output_file = "games_batched.jsonl"  # JSON Lines format

async with BGGClient(token=token) as client:
    with open(output_file, "w") as f:
        for i in range(0, len(game_ids), batch_size):
            batch = game_ids[i : i + batch_size]
            print(f"Fetching batch {i // batch_size + 1}...")
            result = await client.get_thing(batch, stats=True)
            # Write each game as a JSON line
            for game in result.items:
                f.write(json.dumps(game.model_dump()) + "\n")

# Later, read back as needed
import pandas as pd

df = pd.read_json(output_file, lines=True)

Fetching batch 1...
Fetching batch 2...
Fetching batch 3...
Fetching batch 4...
Fetching batch 5...
Fetching batch 6...
Fetching batch 7...
Fetching batch 8...
Fetching batch 9...
Fetching batch 10...
Fetching batch 11...
Fetching batch 12...
Fetching batch 13...
Fetching batch 14...
Fetching batch 15...
Fetching batch 16...
Fetching batch 17...
Fetching batch 18...
Fetching batch 19...
Fetching batch 20...
Fetching batch 21...
Fetching batch 22...
Fetching batch 23...
Fetching batch 24...
Fetching batch 25...
Fetching batch 26...
Fetching batch 27...
Fetching batch 28...
Fetching batch 29...
Fetching batch 30...
Fetching batch 31...
Fetching batch 32...
Fetching batch 33...
Fetching batch 34...
Fetching batch 35...
Fetching batch 36...
Fetching batch 37...
Fetching batch 38...
Fetching batch 39...
Fetching batch 40...
Fetching batch 41...
Fetching batch 42...
Fetching batch 43...
Fetching batch 44...
Fetching batch 45...
Fetching batch 46...
Fetching batch 47...
Fetching batch 48...
F

### Option 3: Build DataFrame Incrementally

In [7]:
import pandas as pd

game_ids = list(range(1, 101))
batch_size = 20
dfs = []

async with BGGClient(token=token) as client:
    for i in range(0, len(game_ids), batch_size):
        batch = game_ids[i : i + batch_size]
        print(f"Fetching batch {i // batch_size + 1}...")
        result = await client.get_thing(batch, stats=True)
        # Convert batch to DataFrame
        batch_df = pd.DataFrame([game.model_dump() for game in result.items])
        dfs.append(batch_df)

# Combine all at once
games_df = pd.concat(dfs, ignore_index=True)

Fetching batch 1...
Fetching batch 2...
Fetching batch 3...
Fetching batch 4...
Fetching batch 5...


### Recommendation: For 1000 games, use Option 2 (stream to disk) - it's most memory-efficient and you can process the data later without re-fetching.

## 4. Get User Collection

In [8]:
username = "eekspider"  # Example user

async with BGGClient(token=token) as client:
    collection = await client.get_collection(username, stats=True)

print(f"User '{username}' has {len(collection.items)} games in their collection\n")
print("First 5 games:")
# for item in collection.items[:5]:
#     print(f"  - {item.name} (Owned: {item.status.own if item.status else 'Unknown'})")

for item in collection.items[:5]:
    owned = item.status.get("own", False) if item.status else False
    print(f"  - {item.name} (Owned: {owned})")

User 'eekspider' has 58 games in their collection

First 5 games:
  - Architekton (Owned: 1)
  - Australian Rails (Owned: 1)
  - Big Boggle (Owned: 1)
  - Boggle Bowl (Owned: 1)
  - British Rails (Owned: 1)


## 5. Get User Plays

In [9]:
username = "eekspider"  # Example user

async with BGGClient(token=token) as client:
    plays = await client.get_plays(username=username)

print(f"User '{username}' has logged {len(plays.plays)} plays\n")
print("Recent plays:")
for play in plays.plays[:5]:
    # print(f"  - {play.item.name} on {play.date}")
    # print(play.item['name'])
    print(f"  - {play.item['name']} on {play.date}")

User 'eekspider' has logged 100 plays

Recent plays:
  - Gang of Four on 2009-03-08
  - Saturn on 2009-03-08
  - Scrabble Me on 2009-03-08
  - Notre Dame on 2009-03-06
  - Dominion on 2009-03-01


## 6. Save Data to Files

In [11]:
# Save search results to JSON
save_json(results.items, "search_results.json")
print("✅ Saved to search_results.json")

# # Save game details to CSV
# save_csv(games.items, "games.csv")
# print("✅ Saved to games.csv")

# Save game details to JSON
# save_json(games.items, "games.json")
# print("✅ Saved to games.json")

# Save large game details to CSV
df.to_csv("large_games.csv")
print("✅ Saved to large_games.csv")

✅ Saved to search_results.json
✅ Saved to large_games.csv


In [12]:
import pandas as pd

games_df = pd.read_csv("large_games.csv")
games_df.head()

,Unnamed: 0,id,type,name,description,yearpublished,minplayers,maxplayers,playingtime,minage,usersrated,average,rank,categories,mechanics,designers,artists,publishers
0,0,1,boardgame,Die Macher,Die Macher is a game about seven sequential po...,1986,3,5,240,14,NaN,NaN,489.0,"['Economic', 'Negotiation', 'Political']","['Alliances', 'Area Majority / Influence', 'Au...",['Karl-Heinz Schmiel'],"['Bernd Brunnhofer', 'Marcus Gschwendtner', 'H...","['Hans im Glück', 'Moskito Spiele', 'Ediciones..."
1,1,2,boardgame,Dragonmaster,Dragonmaster is a trick-taking card game based...,1981,3,4,30,12,NaN,NaN,5459.0,"['Card Game', 'Fantasy']",['Trick-taking'],"['G. W. ""Jerry"" D\'Arcey']",['Bob Pepper'],"['E. S. Lowe Company Inc.', 'Milton Bradley']"
2,2,3,boardgame,Samurai,Samurai is set in medieval Japan. Players comp...,1998,2,4,60,10,NaN,NaN,276.0,"['Abstract Strategy', 'Medieval']","['Area Majority / Influence', 'Enclosure', 'Ha...",['Reiner Knizia'],"['Miguel Valenzuela Beltrán', 'Franz Vohwinkel...","['Fantasy Flight Games', 'Hans im Glück', '999..."
3,3,4,boardgame,Tal der Könige,When you see the triangular box and the luxuri...,1992,2,4,60,12,NaN,NaN,7244.0,['Ancient'],"['Action Points', 'Area Majority / Influence',...",['Christian Beierer'],['Thomas di Paolo'],['KOSMOS']
4,4,5,boardgame,Acquire,"In Acquire, each player strategically invests ...",1963,2,6,90,12,NaN,NaN,359.0,"['Economic', 'Territory Building']","['Connections', 'End Game Bonuses', 'Hand Mana...",['Sid Sackson'],"['Eric Hibbeler', ""Scott O'Gara"", 'Scott Okumu...","['3M', 'The Avalon Hill Game Co', 'Arclight Ga..."


## 7. Extract Raw Text Descriptions

In [13]:
# Get game with full description (Gloomhaven)
async with BGGClient(token=token) as client:
    game = await client.get_thing([174430], stats=True)

if game.items and game.items[0].description:
    description = game.items[0].description
    print(f"Game: {game.items[0].name}")
    print(f"\nDescription ({len(description)} chars):")
    print(description[:500] + "...")
else:
    print("No description available")

Game: Gloomhaven

Description (1576 chars):
Gloomhaven  is a game of Euro-inspired tactical combat in a persistent world of shifting motives. Players will take on the roles of wandering adventurers with their own special sets of skills and their own reasons for traveling to this dark corner of the world. Players must work together out of necessity to clear out menacing dungeons and forgotten ruins. In the process, they will enhance their abilities with experience and loot, discover new locations to explore and plunder, and expand an ever-...


## 8. Data Transformation Example

In [15]:
help(models_to_list)

Help on function models_to_list in module bgg_extractor.transform:

models_to_list(models: collections.abc.Sequence[pydantic.main.BaseModel]) -> list[dict[str, typing.Any]]
    Convert a sequence of Pydantic models to a list of dictionaries.

    Args:
        models: A sequence (list, tuple) of Pydantic model instances.

    Returns:
        A list of dictionaries.



In [18]:
# import pandas as pd

# # Convert to pandas DataFrame
# games_dict = models_to_list(all_games.items)
# df = pd.DataFrame(games_dict)

# print("DataFrame shape:", df.shape)
# print("\nColumns:", df.columns.tolist())
# print("\nFirst few rows:")
# df.head()

## 9. Batch Operations

Get multiple items in one session:

In [19]:
game_ids = [item.id for item in results.items[:5]]

async with BGGClient(token=token) as client:
    # Multiple operations in one session
    all_games = await client.get_thing(game_ids, stats=True)
    user_info = await client.get_user("eekspider")

print(f"Retrieved {len(all_games.items)} games")
print(f"User: {user_info.name}")

Retrieved 5 games
User: eekspider


In [20]:
all_games

ThingSchema(items=[ThingItem(id=134277, type='boardgameexpansion', name='World Wonders (fan expansion for Catan)', description='The Island of Catan is rich with resources and beauty. The great nations meet upon the island for a decade-long festival of culture and trade... and a grand competition. Living solely off of the land, they must build the greatest society they can in a short time. To do this, the nations decide to recreate the Wonders of the Ancient World. But only one of each Wonder can exist, and there are only so many Wonders that the land can support!\n\nThis expansion, which has been in development for over 2 years, can be combined with The Settlers of Catan base game or with many of Catan: Seafarers and Catan: Traders & Barbarians scenarios. As you upgrade your Wonders, you will gain abilities that allow you to either rake in the resources or wreck havoc on your opponents. Combine these abilities in a game that will have you revisiting Catan as if you have never really pl

In [21]:
user_info

UserSchema(id=212736, name='eekspider', firstname='Karen', lastname='Lee', avatar=None, registered='2008', buddies=[], guilds=[], hot=[], top=[])